In [29]:
import pandas as pd
import regex as re
from sklearn.utils import resample
import warnings 
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import re 

# some insights of data 
text given in each datapoint is a tweet or grabage values with is in many languages.
the data is given in 4 files with label as the file name.
as machine can be trained by only numerical values , we have to convert it to vectors ,before that all the text should be in same language ,
so we can translate the text from non-english to english and train the machine or by dropping tweets which are in non-english language


In [2]:
litigious_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\litigious.csv",encoding='latin-1')
negative_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\negative.csv",encoding='latin-1')
positive_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\positive.csv",encoding='latin-1')
uncertainty_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\uncertainty.csv",encoding='latin-1')

In [3]:
litigious_data['new_column']='litigious'
negative_data['new_column']='negative'
positive_data['new_column']='positive'
uncertainty_data['new_column']='uncertainty'

As the label is given as name of csv file i have added a column with column name as new_column for every dataframe

In [4]:
data = pd.concat([litigious_data,negative_data,positive_data,uncertainty_data])

concatinating all the 4 dataframes in to one 

In [8]:
data = data.reset_index()

reseting all the index values as we have concatinated many dataframes to one 

In [18]:
data.drop(labels=['index','Unnamed: 0'],axis=1,inplace=True)

removing unwanted columns

In [19]:
data['Language'].value_counts()

en                                                                                                                                                                      871310
fr                                                                                                                                                                       13091
es                                                                                                                                                                       11333
pt                                                                                                                                                                       10336
ja                                                                                                                                                                        8414
                                                                                                                             

As the majority of the language tweets are of english (~93%),it takes much time to translate non-english tweet to english, we can remove remaining rows which are in other language

In [21]:
data = data[data['Language']=='en']

only enlgish language tweets are present in the dataframe so we can do conversion to vector easily 

In [26]:
data

,Text,Language,new_column
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
2,OOPS. I typed her name incorrectly (todayâs ...,en,litigious
3,@SaltBurned [When the first sign of surrender ...,en,litigious
4,"""It's Your fight to save the UK from the liars...",en,litigious
...,...,...,...
937849,There is a bag of cheese hidden somewhere in m...,en,uncertainty
937850,@Treg2Cole Thatâs a hellz nah for me. ð¤¢. ...,en,uncertainty
937851,@Sam_Mallory El Mizouni and Roles in the same ...,en,uncertainty
937852,If you donât take risks youâll always end ...,en,uncertainty


In [28]:
data.isnull().sum()

Text          0
Language      0
new_column    0
dtype: int64

In [136]:
stop_words = stopwords.words("english")
stop_words.remove('not')

this stop words which occur often and meaning of sentance will not be changed if they are not present , so that dimensions of the vector can be reduced , not is removed assuming that if that is removed from the sentance the meaning could be reversed  

In [145]:
%%time
sentances = []
for i in data['Text']:
    word = []
    sent = i.lower()
    sent = emoji.demojize(sent)
    sent = re.sub('â€˜|â€™',"'",sent)
    sent = re.sub('&amp;',"and",sent)
    sent = re.sub('<.*?>','',sent)
    sent = re.sub('https?://\S*|www\S*','',sent)
    sent = re.sub(' @\w+| #\w+',"",sent)
    sent = re.sub("[^a-z0-9 ]","",sent)
    for i in word_tokenize(sent):
        if i in stop_words:pass
        else:word.append(PorterStemmer().stem(i))
    sentances.append(" ".join(word))

Wall time: 13min 6s


time taken to do porterstemmer on 871310 rows is 13min 6sec, in this step we have done text pre-processing , this all the steps inside are done for redcution of vector dimensions


In [150]:
cleaned_text_data = pd.DataFrame({"cleaned_Text":sentances})

In [151]:
cleaned_text_data

,cleaned_Text
0,charliecorley testimoni evid court law state f...
1,flagstar bank disclos data breach impact 15mil...
2,oop type name incorrectli today brave wit 6 7 ...
3,saltburn first sign surrend come michael smile...
4,fight save uk liar cheat charlatan steve bray ...
...,...
871305,bag chees hidden somewher hous aint talk
871306,treg2col that hellz nah mayb fri
871307,sammallori el mizouni role team could realli g...
871308,dont take risk youll alway end work someon kin...


creating dataframe of each data piont in text column 

In [166]:
#bag of words
count_obj = CountVectorizer()
trans = count_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:25000])
trans

<25000x42933 sparse matrix of type '<class 'numpy.int64'>'
	with 347710 stored elements in Compressed Sparse Row format>

there are to 42933 unique words 

In [170]:
array_count = trans.toarray()

In [171]:
count_obj.vocabulary_

{'charliecorley': 8158,
 'testimoni': 37684,
 'evid': 13930,
 'court': 9969,
 'law': 22291,
 'state': 36152,
 'feder': 14557,
 'must': 26164,
 'stand': 36080,
 'cross': 10299,
 'examin': 13980,
 'flagstar': 14930,
 'bank': 5082,
 'disclos': 11953,
 'data': 10836,
 'breach': 6607,
 'impact': 18854,
 '15million': 439,
 'individu': 19085,
 'oop': 28074,
 'type': 39632,
 'name': 26325,
 'incorrectli': 19011,
 'today': 38662,
 'brave': 6584,
 'wit': 41861,
 'probabi': 30485,
 'dont': 12357,
 'forget': 15197,
 'request': 32142,
 'saltburn': 33379,
 'first': 14854,
 'sign': 34908,
 'surrend': 36916,
 'come': 9093,
 'michael': 24962,
 'smile': 35334,
 'split': 35882,
 'face': 14241,
 'like': 22938,
 'sharp': 34575,
 'knife': 21740,
 'flesh': 14964,
 'would': 42174,
 'chill': 8337,
 'dean': 11072,
 'core': 9778,
 'ice': 18590,
 'cold': 8999,
 'exist': 14059,
 'natur': 26446,
 'archangel': 4115,
 'enj': 13461,
 'fight': 14732,
 'save': 33614,
 'uk': 39727,
 'liar': 22823,
 'cheat': 8198,
 'charl

this command gives all the unique words with there index value in the vector 

In [172]:
pd.DataFrame(array_count,columns=sorted(count_obj.vocabulary_))

,00,000,000001,0007,001,0047,004coinmarketcap,006,007,007anoth,...,zumwalt,zuneth,zuwairahnasir,zyagle09,zychaxrma,zygotes3,zylstra,zzztyler,zzzz,zzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
#TF-IDF
tf_obj = TfidfVectorizer()
trans_tf = tf_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:5000])
trans_tf

<5000x14163 sparse matrix of type '<class 'numpy.float64'>'
	with 68552 stored elements in Compressed Sparse Row format>

In [215]:
pd.DataFrame(trans_tf.toarray(),columns=sorted(tf_obj.vocabulary_.keys()))

,0047,007,01,02906for,030,033,036,038crypto,042,05,...,zoomcarindia,ztpetrizzo,zu,zubac,zubair,zuneth,zuwairahnasir,zychaxrma,zygotes3,zylstra
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


in countvectorizer (bag of words) and tfidfvectorizer(TF-IDF) i have reduced datapoints because of memory problem so i have limited to 25000 rows in countvectorizer instead of 871310 and 5000 in tfidfvectorizer 